In [1]:
import h5py
import pandas as pd
import numpy as np
from scipy import stats
import os
import json
from glob import glob
import zipfile
from collections import OrderedDict
from operator import itemgetter
from pga.return_pid import return_pid
from datetime import datetime
import numpy_indexed as npi
from tqdm import tqdm

In [49]:
os.chdir('/home/valesco/Datasets/PGA_Data/lineup_combos')
file = h5py.File('pga_5_team_combos.h5', 'r')

file_keys = file.keys()

combos = np.array(file['batch1_0'])
#combos = combos[1:,:5]

In [50]:
combos = combos[np.where((combos < 30).all(axis = 1))]
combos.shape

(175310, 5)

In [4]:
os.chdir('/home/valesco/Documents/')

sal_df = pd.read_csv('DKSalaries (52).csv', skiprows = 7, usecols = range(7,14))
sal_df.reset_index(inplace = True, drop = True)
sal_df.columns = ['pos', 'name_id', 'player_name', 'id', 'salary', 'tournament', 'team']
sal_df.head()
#sal_df = sal_df.loc[sal_df['player_name'] != 'Stewart Hagestad']
#sal_df = sal_df.loc[sal_df['player_name'] != 'Toto Gana']

,pos,name_id,player_name,id,salary,tournament,team
0,G,Dustin Johnson (9048882),Dustin Johnson,9048882,12000,The Open Championship,Golf
1,G,Jordan Spieth (9048883),Jordan Spieth,9048883,11600,The Open Championship,Golf
2,G,Jon Rahm (9048884),Jon Rahm,9048884,11400,The Open Championship,Golf
3,G,Rory McIlroy (9048885),Rory McIlroy,9048885,11000,The Open Championship,Golf
4,G,Sergio Garcia (9048886),Sergio Garcia,9048886,10600,The Open Championship,Golf


In [6]:
pid_ls = []
for player in sal_df['player_name'].unique():
    pid = return_pid(player)
    
    if pid is not np.nan:
        pid_ls.append(pid)
    else:
        pid_ls.append(-999)
        print(player, pid)
        
sal_df['pid'] = pid_ls   
sal_df = sal_df[sal_df['pid'] > 0]

Connor Syme nan
Harry Ellis nan
Joe Dean nan
Alfie Plant nan
Adam Hodkinson nan
Nick McCarthy nan


In [7]:
sal_df['pid'] = sal_df['pid'].apply(lambda x: int(x))
sal_df.sort_values(by = 'pid', inplace = True)
sal_df['temp_pid'] = sal_df['pid'].rank(ascending = True)
sal_df['temp_pid'] = sal_df['temp_pid'].apply(lambda x: int(x - 1))
sal_df['sal/100'] = sal_df['salary'] / 100
sal_df['sal/100'] = sal_df['sal/100'].apply(lambda x: int(x))
sal_df['sal_rank'] = sal_df['salary'].rank(ascending = False)
sal_df['sal_rank'] = sal_df['sal_rank'].apply(lambda x: int(x))
sal_df.head()

,pos,name_id,player_name,id,salary,tournament,team,pid,temp_pid,sal/100,sal_rank
125,G,John Daly (9049007),John Daly,9049007,6100,The Open Championship,Golf,1249,0,61,129
145,G,Todd Hamilton (9049027),Todd Hamilton,9049027,6000,The Open Championship,Golf,1473,1,60,141
132,G,Tom Lehman (9049014),Tom Lehman,9049014,6100,The Open Championship,Golf,1677,2,61,129
147,G,Sandy Lyle (9049029),Sandy Lyle,9049029,6000,The Open Championship,Golf,1717,3,60,141
12,G,Phil Mickelson (9048894),Phil Mickelson,9048894,8700,The Open Championship,Golf,1810,4,87,13


In [40]:
sal_dict = dict(zip(np.array(sal_df['temp_pid'].values).astype(np.int16), 
                    np.array(sal_df['sal/100'].values).astype(np.int16)))

top_df = sal_df.loc[sal_df['sal_rank'] <= 10]
rank_dict = dict(zip(np.array(top_df['temp_pid'].values).astype(np.int16),
                     np.array(top_df['sal/100'].values).astype(np.int16)))

In [45]:
#test_combos = npi.unique(combos[:-2999])
start = datetime.now()

#combos = combos[1:,:5]
#combos = npi.unique(combos)
#combos = combos[np.where((combos < 30).all(axis = 1))]

sal_array = np.vectorize(sal_dict.__getitem__)(combos)

print(datetime.now() - start)

#test_combos[np.where((test_combos == (1,2,3,4,5,1)).all(axis = 1))]
sal_array

0:00:01.167091


array([[ 61,  60,  61,  60,  87],
       [ 61,  60,  61,  60,  60],
       [ 61,  60,  61,  60,  65],
       ..., 
       [ 77, 103,  60,  63,  72],
       [ 77, 103,  69,  63,  72],
       [ 77,  60,  69,  63,  72]], dtype=int16)

In [46]:
final_array = np.zeros(shape = (1,6)).astype(np.int16)

for pid, salary in tqdm(rank_dict.items()):
    
    temp_sal = np.full((len(sal_array), 1), salary)

    sal_array = np.hstack((temp_sal, sal_array))
    indices = np.where((sal_array.sum(axis = 1) < 501) & (sal_array.sum(axis = 1) > 450))

    sal_array = sal_array[:,1:]

    temp_combo = combos[indices]
    temp_pid = np.full((len(temp_combo), 1), pid, np.int16)
    temp_combo = np.hstack((temp_pid, temp_combo))

    vals, counts = stats.mode(temp_combo, axis = 1)

    temp_array = np.hstack((temp_combo, counts)).astype(np.int16)
    temp_array = temp_array[temp_array[:,6] == 1]
    temp_array = temp_array[:,:-1]

    final_array = np.vstack((final_array, temp_array))

100%|██████████| 10/10 [00:00<00:00, 14.35it/s]


In [47]:
sal_array[np.where((sal_array.sum(axis = 1) < 501) & (sal_array.sum(axis = 1) > 450))]

array([[ 61,  87, 106,  96, 103],
       [ 60,  87, 106,  96, 103],
       [ 61,  87, 106,  96, 103],
       [ 60,  87, 106,  96, 103],
       [ 87,  60, 106,  96, 103],
       [ 87,  65, 106,  96, 103],
       [ 87,  69, 106,  96, 103],
       [ 87,  61, 106,  96, 103],
       [ 87,  60, 106,  96, 103],
       [ 87,  67, 106,  96, 103],
       [ 87,  60, 106,  96, 103],
       [ 87,  63, 106,  96, 103],
       [ 87,  63, 106,  96, 103],
       [ 87,  77, 106,  96, 103],
       [ 87,  62, 106,  96, 103],
       [ 87,  72, 106,  96, 103],
       [ 87, 106,  67,  96, 103],
       [ 87, 106,  96,  66, 103],
       [ 87, 106,  96,  68, 103],
       [ 87, 106,  96,  62, 103],
       [ 87, 106,  96,  62, 103],
       [ 87, 106,  96,  62, 103],
       [ 87, 106,  96, 103,  60],
       [ 87, 106,  96, 103,  69],
       [ 87, 106,  96, 103,  63],
       [ 87, 106,  96, 103,  72],
       [ 87,  67, 106,  96, 103],
       [ 87,  60, 106,  96, 103],
       [ 87,  63, 106,  96, 103],
       [ 87,  

In [133]:
final_array.shape

(65311001, 6)

In [134]:
start = datetime.now()
test_sal_array = np.vectorize(sal_dict.__getitem__)(final_array[1:])
print(datetime.now() - start)

0:09:40.980470


In [143]:
sal_sum = test_sal_array.sum(axis = 1).reshape(len(test_sal_array), 1)
np.hstack((test_sal_array, sal_sum))

array([[ 95,  63,  64, ...,  87,  83, 452],
       [ 95,  63,  64, ...,  87,  92, 461],
       [ 95,  63,  64, ...,  87,  88, 457],
       ..., 
       [115,  60,  73, ...,  86,  62, 470],
       [115,  60,  73, ...,  62,  62, 458],
       [115,  60,  74, ...,  62,  62, 459]])

In [136]:
sal_sum.shape

(65311000,)

In [137]:
test_sal_array.shape

(65311000, 6)

In [142]:
sal_sum

array([451, 451, 451, ..., 500, 500, 500])

In [141]:
sal_sum.reshape(len(test_sal_array), 1)

array([[451],
       [451],
       [451],
       ..., 
       [500],
       [500],
       [500]])